# TP2 : Extraction d'informations

Sur la base des éléments méthodologiques et des enseignements techniques présentés lors du cours théorique, il est demandé dans le cadre de ce TP :
- de se familiariser avec les techniques d’enrichissement (extraction de mots-clés, reconnaissance d’entités nommées, analyse de sentiment...)
- d’appliquer celles-ci au corpus de bulletins de la ville de Bruxelles préalablement convertis au format .txt

Pour ce faire, vous utiliserez différentes librairies Python vues au cours.

## 1. Keywords

### Librairies utilisées dans le code

#### Choix d'une année à analyser

In [1]:
import os
from collections import defaultdict, Counter
import textract
import nltk
from nltk.corpus import stopwords
import yake
from wordcloud import WordCloud
from IPython.display import Image

### Sélection des fichiers à analyser

#### Choix d'une année à analyser

In [2]:
selected_year = 1939

#### Stocker la liste des fichiers contenus dans le dossier 'txt'

In [3]:
txt_path = "../data/txt/"
files = sorted(os.listdir(txt_path))
len(files)

2827

#### Lister les fichiers pour Bruxelles et pour Laeken

In [4]:
bxl_selected_year = [f for f in files if f.startswith('Bxl_') and str(selected_year) in f]
print(len(bxl_selected_year))

lkn_selected_year = [f for f in files if f.startswith('Lkn_') and str(selected_year) in f]
print(len(lkn_selected_year))

24
0


#### Vérification du nombre de fichiers

In [5]:
count_years = defaultdict(int)

for f in files: 
    if "_" in f and f.endswith("txt"): 
        elems = f.split("_") 
        year = elems[1] 
        count_years[year] += 1 
        
n_selected_year = count_years[str(selected_year)]
print(n_selected_year)

24


### Récupérer le texte des fichiers

#### Créer le dossier adéquat

In [6]:
path_year = os.path.join('../data/', str(selected_year))
if not os.path.exists(path_year):
    os.mkdir(path_year)

#### Copier les fichiers désirés dans le dossier

In [7]:
for txt in bxl_selected_year:
    text = textract.process(os.path.join(txt_path, txt))
    with open(os.path.join(path_year, txt), 'wb') as f: 
        f.write(text) 

#### Concaténer le texte des fichiers

In [8]:
!cat ../data/{selected_year}/*.txt > ../data/{selected_year}.txt

#### Compter le nombre de mots repris dans le fichier

In [9]:
!wc -m ../data/{selected_year}.txt

3642058 ../data/1939.txt


#### Vérification du nombre de mots

In [10]:
text_selected = open(os.path.join(f'../data/{selected_year}.txt'), 'r').read()
print(len(text_selected))

3642058


#### Affichage des premiers caractères

In [11]:
text_selected[:500]

"\x0c—\nNo 5.\n\n859 —\n\nC O M P T E R E N D U D E L A S E A N C E D U 23 O C T O B R E 1939.\n\nVILLE DE BRUXELLES\n\nBULLETIN COMMUNAL\nAnnée 1939\nCONSEIL\nSéance\n\nCOMMUNAL\n\ndu 23 octobre\n\n1939.\n\nP r é s i d e n c e de M . A D O L P H E M A X , Bourgmestre.\n\nSOMMAIRE :\nPAGES.\n\n1.\n\nCommunications\n\n2.\n\nProjet d'unification des règlements généraux de police dans les diverses communes de l'agglomération\nbruxelloise\nAdoption. 861\n\n3.\n\nEglise anglicane du Christ, à Ixelles. — Compte de\n1938\nAvis favorable.\nEglise"

### Extraction des mots clés

#### Extraire les mots clés pour l'année entière

In [12]:
kw_extractor = yake.KeywordExtractor(lan="fr", top=100)
text_all = open(os.path.join(f'../data/{selected_year}.txt'), 'r').read()
keywords = kw_extractor.extract_keywords(text_all.lower())

ignored = ["conseil communal", "conseil général", "conseiller communal", "rue rue", "avenue rue", "maison rue"]

kw_results = []
unigr = []
for kw, score in keywords:
    words = kw.split()
    if len(words) == 2 and kw.lower() not in ignored:
        kw_results.append(kw.lower())
    if len(words) == 1 and kw.lower() not in ignored:
        unigr.append(kw.lower())

print(sorted(set(kw_results)))

['avis favorable', 'budget extraordinaire', 'caisse publique', "commission d'assistance", 'crédit supplémentaire', "d'assistance publique", "d'autre part", "d'un crédit", 'het collège', "l'administration communale", "l'echevin coelst", "l'echevin van", 'rue van', 'séance publique', 'van den', 'van halteren', 'van het', 'van remoortel', 'voie publique', 'voies publiques', "vote d'un"]


#### Ajout de mots à ignorer dans l'analyse des fichiers un à un

In [13]:
ignored += kw_results

#### Analyse séquentielle des fichiers de l'année

In [14]:
kw_extractor = yake.KeywordExtractor(lan="fr", top=20)

kw_details = []

for f in bxl_selected_year:
    text = open(os.path.join(path_year, f), 'r').read()
    keywords = kw_extractor.extract_keywords(text.lower())
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2 and kw.lower() not in ignored:
            kw_details.append(kw)
        if len(words) == 1 and kw.lower() not in ignored:
            unigr.append(kw)
            
print(sorted(set(kw_details)))

['avenue avenue', 'chiffre électoral', "conseil d'administration", 'cours cours', 'credits depenses', 'crédits supplémentaires', 'depenses faites', "frais d'entretien", 'frais généraux', 'francs francs', 'het verboden', 'hoogte van', "l'année scolaire", "l'hygiène publique", "l'hôpital brugmann", 'loi communale', "nombre d'élèves", 'nomination définitive', 'nomination provisoire', 'pensions communales', 'personnel adoption', 'personnel subalterne', 'section section', 'service administratif', 'service intérieur', 'service médical', 'société royale', "travaux d'entretien", 'voor het', 'votes nominatifs', 'écoles libres']


In [15]:
kw_extractor = yake.KeywordExtractor(lan="fr", top=30)

kw_details = []

for f in bxl_selected_year:
    text = open(os.path.join(path_year, f), 'r').read()
    keywords = kw_extractor.extract_keywords(text.lower())
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2 and kw.lower() not in ignored:
            kw_details.append(kw)
        if len(words) == 1 and kw.lower() not in ignored:
            unigr.append(kw)
            
print(sorted(set(kw_details)))

['année année', "année d'études", 'assche bruxelles', 'avenue avenue', "chiffre d'éligibilité", 'chiffre électoral', 'commerces multiples', "conseil d'administration", 'conseiller prestation', 'cours cours', 'credits depenses', 'crédits supplémentaires', "d'une pension", "d'école primaire", 'den openbaren', 'depenses faites', 'divisions territoriales', 'door het', "frais d'entretien", 'frais généraux', 'francs francs', 'groupes transformateurs', 'het verboden', 'hoogte van', 'hôpital saint-pierre', "l'année scolaire", "l'echevin catteau", "l'echevin verheven", "l'hygiène publique", "l'hôpital brugmann", 'loi communale', 'musée communal', 'naar den', 'naissances vivantes', "nombre d'élèves", 'nombre total', 'nomination définitive', 'nomination provisoire', 'pensions communales', 'personnel administratif', 'personnel adoption', 'personnel médical', 'personnel subalterne', 'population présente', 'présent règlement', 'quotient électoral', 'rue neuve', 'section section', 'service administra

In [16]:
print(sorted(set(unigr)))

['action', 'adoption', 'adoptés', 'affaire', 'agents', 'analyses', 'année', 'art', 'articles', 'ateliers', 'autorisation', 'avenue', 'avis', 'belge', 'belges', 'boulevard', 'bourgmestre', 'bourgmestre.', 'brugmann', 'brunfaut', 'bruxelles', 'budget', "c'est", 'cabinet', 'caisse', 'carnaval', 'cas', 'chiffre', 'coelst', 'collège', 'commerces', 'commission', 'communal', 'communale', 'communales', 'compte', 'compteurs', 'conseil', 'conseiller', 'construction', 'contrôle', 'cours', 'crédit', 'crédits', "d'assistance", "d'entretien", "d'un", "d'une", "d'école", 'dat', 'demande', 'den', 'depenses', 'der', 'die', 'direction', 'district', 'division', 'divisions', 'don', 'décembre', 'décès', 'défendu', 'définitive', 'démission', 'dépenses', 'ecole', 'ecoles', 'enfants', 'entretien', 'envoi', 'faire', 'favorable', 'fonds', 'frais', 'francs', 'février', 'gaz', 'groupe', 'habitants', 'het', 'heures', 'hôpital', 'infirmières', 'installation', 'interdit', 'janvier', 'juillet', "l'année", "l'article"

### Nettoyage des fichiers pour une analyse supplémentaire

#### Création de la 'stopwrods' enrichie

In [17]:
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout", 
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu"]
sw += ignored
sw += unigr

#### Fonction de nettoyage du fichier de l'année considérée

In [18]:
def clean_text(selected_year):
    input_path = f"../data/{selected_year}.txt"
    output_path = f"../data/{selected_year}_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

In [19]:
clean_text(selected_year)

'Output has been written in ../data/1939_clean.txt!'

#### Récupérer les mots les plus fréquents

In [20]:
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)
text_cleaned = open(os.path.join(f'../data/{selected_year}_clean.txt'), 'r').read()
keywords = kw_extractor.extract_keywords(text_cleaned.lower())

kw_results = []

for kw, score in keywords:
    words = kw.split()
    if len(words) == 3 and kw.lower() not in ignored:
        kw_results.append(kw.lower())

print(sorted(set(kw_results)))

['allocation prévue article', 'appel nominal adoptées', 'appel nominal prennent', 'article figurant tableau', 'brunet derudder jardin', 'carton wiart piron', 'catteau swolfs verhaeghe', 'conclusions rapports mises', 'demuyter xavier carton', 'derudder jardin gérard', 'déduction faite somme', 'echevin nom rapports', 'echevin verheven nom', 'exercice extraordinaire supplémentaires', 'exercice ordinaire supplémentaires', 'faite somme versée', 'gérard relecom dispy', 'honneur proposer émettre', 'institutrice école primaire', 'jardin gérard relecom', 'lieu décider supplémentaire', 'marteaux vermeire moyson', 'meulebroeck catteau swolfs', 'mises voix appel', 'molenbeek saint jean', 'moyson thielemans remoortel', 'naeyer verheven waucquez', 'observation honneur proposer', 'produit vente terrains', 'propose supplémentaire couvert', 'proposer émettre approbation', 'proposer émettre assistance', 'proposer émettre eglise', 'prévue article figurant', 'rapports mises voix', 'saint gilles saint', 's

#### Agrandissement de la liste ignorée

In [21]:
ignored += kw_results

#### Création d'un répertoire pour les fichiers nettoyés

In [22]:
path_cleaned = os.path.join('../data/cleaned/')
if not os.path.exists(path_cleaned):
    os.mkdir(path_cleaned)

#### Fonction de nettoyage pour les fichiers de l'année considérée

In [23]:
def clean_files(selected_year):
    for f in bxl_selected_year:
        input_path = f"../data/{selected_year}/{f}"
        output_path = f"../data/cleaned/{f}"
        output = open(output_path, "w", encoding='utf-8')
        with open(input_path, encoding='utf-8') as f:
            text = f.read()
            words = nltk.wordpunct_tokenize(text)
            kept = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
            kept_string = " ".join(kept)
            output.write(kept_string)
    return f'Final output has been written in {output_path}!'

In [24]:
clean_files(selected_year)

'Final output has been written in ../data/cleaned/Bxl_1939_Tome_I_Part_9.txt!'

#### Test de la méthode appliquée aux fichiers pris un à un

In [25]:
kw_extractor = yake.KeywordExtractor(lan="fr", top=5)

kw_details = []

for f in bxl_selected_year:
    text = open(os.path.join(path_cleaned, f), 'r').read()
    keywords = kw_extractor.extract_keywords(text.lower())
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 3 and kw.lower() not in ignored:
            kw_details.append(kw)
            
print(sorted(set(kw_details)))

['académie beaux arts', 'alloués demandés arrêtés', 'article adopté chapitre', 'assemblée générale fixe', 'autorisations délivrées nature', 'avertissement extrait rôle', 'aveugles refuge ursulines', 'base journée présence', 'blanchissage nettoyage chauffage', 'cambre saint philippe', 'charles buis primaires', 'civil période renouvelable', 'concession temporaire catégorie', 'coopérative intercommunale crémation', 'coopérative maison peuple', 'cooremans losange mattys', 'credits faites alloués', 'crèmes glacées glaces', 'côté numéros pairs', 'disponibilité motif santé', 'délivrées nature etablissements', 'een vergunning burgemeester', 'eglise dame finistère', 'eleves habitant faubourgs', 'faite concession temporaire', 'faites alloués demandés', 'fêtes port masque', 'gertrude hospice aveugles', 'gilles saint josse', 'groupes transformateurs transformateur', 'habitant faubourgs province', 'hectares ares centiares', 'honorable echevin finances', 'hospice aveugles refuge', 'illégitimes mascu

#### Tokenisation

In [ ]:
words = nltk.wordpunct_tokenize(text_selected)
print(f"{len(words)} words found")

print(words[:200])

#### Calculer la taille du vocabulaire

In [ ]:
words_cleaned = []

for w in words:
    if len(w) > 2 and w.isalpha() and w.lower() not in sw:
        words_cleaned.append(w.lower())
        
voc = set(words_cleaned)

print(f"{len(words_cleaned)} words kept ({len(voc)} different word forms)")
print(words_cleaned[:200])

OU

In [ ]:
words_cleaned = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
voc = set(words_cleaned)
print(f"{len(words_cleaned)} words kept ({len(voc)} different word forms)")

In [ ]:
fdist = nltk.FreqDist(words_cleaned)
fdist.most_common(10)

#### Ajout d'une liste de mots à ignorer

In [ ]:
sw += ["francs", "ville", "bruxelles", "conseil", "service", "conseil"] 

print(sorted(sw))

In [ ]:
sample_1 = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
voc = set(sample_1)
print(f"{len(sample_1)} words kept ({len(voc)} different word forms)")

In [ ]:
fdist = nltk.FreqDist(sample_1)
fdist.most_common(10)

In [ ]:
sw += ["collège", "communal", "bourgmestre"] 

print(sorted(sw))

In [ ]:
sample_2 = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
voc = set(sample_2)
print(f"{len(sample_2)} words kept ({len(voc)} different word forms)")

In [ ]:
fdist = nltk.FreqDist(sample_2)
fdist.most_common(20)

In [ ]:
sw += ["total", "echevin", "membres", "divers", "année", "article", "question", "diverses", "messieurs", "art"] 

print(sorted(sw))

### Enrichissement de la liste des stopwords

## 2. Named Entity Recognition